In [78]:
import numpy as np
import argparse
import time
import os
import collections
import json

from utils.data_utils import load_dataset_numpy

import scipy.spatial.distance

In [79]:
parser = argparse.ArgumentParser()
parser.add_argument("--dataset_in", default='MNIST',
                    help="dataset to be used")
parser.add_argument("--norm", default='l2',
                    help="norm to be used")
parser.add_argument('--num_samples', type=int, default=None)
parser.add_argument('--n_classes', type=int, default=2)
parser.add_argument('--eps', type=float, default=None)
parser.add_argument('--approx_only', dest='approx_only', action='store_true')
parser.add_argument('--use_test', dest='use_test', action='store_true')
parser.add_argument('--track_hard', dest='track_hard', action='store_true')
parser.add_argument('--new_marking_strat', type=str, default=None)

_StoreAction(option_strings=['--new_marking_strat'], dest='new_marking_strat', nargs=None, const=None, default=None, type=<class 'str'>, choices=None, help=None, metavar=None)

In [80]:
args = parser.parse_args("--dataset_in=MNIST --num_samples=2000".split())

In [81]:
train_data, test_data, data_details = load_dataset_numpy(args, data_dir='data',
														training_time=False)
DATA_DIM = data_details['n_channels']*data_details['h_in']*data_details['w_in']

X = []
Y = []

# Pytorch normalizes tensors (so need manual here!)
if args.use_test:
	for (x,y,_, _, _) in test_data:
		X.append(x/255.)
		Y.append(y)
else:
	for (x,y,_, _, _) in train_data:
		X.append(x/255.)
		Y.append(y)

X = np.array(X)
Y = np.array(Y)

num_samples = int(len(X)/2)
print(num_samples)

if not os.path.exists('distances'):
	os.makedirs('distances')

if 'MNIST' in args.dataset_in or 'CIFAR-10' in args.dataset_in:
	class_1 = 3
	class_2 = 7
	if args.use_test:
		dist_mat_name = args.dataset_in + '_test_' + str(class_1) + '_' + str(class_2) + '_' + str(num_samples) + '_' + args.norm + '.npy'
	else:
		dist_mat_name = args.dataset_in + '_' + str(class_1) + '_' + str(class_2) + '_' + str(num_samples) + '_' + args.norm + '.npy'
	X_c1 = X[:num_samples].reshape(num_samples, DATA_DIM)
	X_c2 = X[num_samples:].reshape(num_samples, DATA_DIM)
	if os.path.exists(dist_mat_name):
		D_12 = np.load('distances/' + dist_mat_name)
	else:
		if args.norm == 'l2':
			D_12 = scipy.spatial.distance.cdist(X_c1,X_c2,metric='euclidean')
		elif args.norm == 'linf':
			D_12 = scipy.spatial.distance.cdist(X_c1,X_c2,metric='chebyshev')
		np.save('distances/' + dist_mat_name, D_12)


2000


In [82]:
if args.norm == 'l2' and 'MNIST' in args.dataset_in:
	# eps_list = np.linspace(3.2,3.8,4)
	eps_list = np.linspace(3.8,3.8,1)
	# eps_list=[2.6,2.8]
elif args.norm == 'l2' and 'CIFAR-10' in args.dataset_in:
	eps_list = np.linspace(4.0,10.0,13)
elif args.norm == 'linf' and 'MNIST' in args.dataset_in:
	eps_list = np.linspace(0.1,0.5,5)
elif args.norm == 'linf' and 'CIFAR-10' in args.dataset_in:
	eps_list = np.linspace(0.1,0.5,5)

if args.eps is not None:
	eps_list = [args.eps]

print(eps_list)

[3.8]


In [83]:
for eps in eps_list:
	print(eps)
    # Add edge if cost 0
	edge_matrix = D_12 <= 2*eps
	edge_matrix = edge_matrix.astype(float)

3.8


In [92]:
len(np.where(edge_matrix!=0)[1])

35055

In [43]:
n_1 = args.num_samples
n_2 = args.num_samples

In [44]:
from scipy.sparse import csr_matrix
from scipy.sparse.csgraph import maximum_flow
# import pyximport
# pyximport.install()
from utils.flow import maximum_flow_custom
import numpy as np

In [3]:
n_1 = 9
n_2 = 9

In [4]:
edge_matrix=np.zeros((n_1,n_2))
edge_matrix[0,0]=1
edge_matrix[1,0]=1
edge_matrix[2,1]=1
edge_matrix[3,1]=1
edge_matrix[3,2]=1
edge_matrix[4,2]=1
edge_matrix[5,3]=1
edge_matrix[6,4]=1
edge_matrix[7,5]=1
edge_matrix[7,6]=1
edge_matrix[7,7]=1

In [45]:
graph_rep = []
for i in range(n_1+n_2+2):
    graph_rep.append([])
    if i==0:
        #source
        for j in range(n_1+n_2+2):
            if j==0:
                graph_rep[i].append(0)
            elif 1<=j<=n_1:
                graph_rep[i].append(n_2)
            elif n_1<j<=n_1+n_2+1:
                graph_rep[i].append(0)
    elif 1<=i<=n_1:
        # LHS vertices
        for j in range(n_1+n_2+2):
            if j<=n_1:
                graph_rep[i].append(0)
            elif n_1<j<=n_1+n_2:
                if edge_matrix[i-1,j-n_1-1]:
                    graph_rep[i].append(n_1*n_2)
                else:
                    graph_rep[i].append(0)
            elif n_1+n_2<j:
                graph_rep[i].append(0)
    elif n_1<i<=n_1+n_2:
        #RHS vertices
        for j in range(n_1+n_2+2):
            if j<=n_1+n_2:
                graph_rep[i].append(0)
            elif j>n_1+n_2:
                graph_rep[i].append(n_1)
    elif i==n_1+n_2+1:
        #Sink
        for j in range(n_1+n_2+2):
            graph_rep[i].append(0)

In [46]:
graph_rep_array=np.array(graph_rep)

In [ ]:
graph_rep_array

In [47]:
graph=csr_matrix(graph_rep_array)

In [48]:
import time

In [49]:
time1=time.time()
A=maximum_flow(graph, 0, n_1+n_2+1)
time2=time.time()

In [50]:
A.flow_value

956000

In [51]:
time3=time.time()
B=maximum_flow_custom(graph, 0, n_1+n_2+1)
time4=time.time()

In [61]:
B.flow_value

956000

In [62]:
time_cython=time2-time1
time_python=time4-time3

In [63]:
print(time_cython,time_python)

0.06453132629394531 47.61944890022278


In [64]:
B.path_edges

[(0, 1),
 (0, 2),
 (0, 3),
 (0, 4),
 (0, 7),
 (0, 8),
 (0, 10),
 (0, 12),
 (0, 13),
 (0, 15),
 (0, 17),
 (0, 18),
 (0, 19),
 (0, 20),
 (0, 21),
 (0, 22),
 (0, 24),
 (0, 25),
 (0, 26),
 (0, 27),
 (0, 29),
 (0, 30),
 (0, 31),
 (0, 32),
 (0, 34),
 (0, 35),
 (0, 36),
 (0, 37),
 (0, 38),
 (0, 40),
 (0, 41),
 (0, 42),
 (0, 43),
 (0, 44),
 (0, 45),
 (0, 47),
 (0, 48),
 (0, 49),
 (0, 50),
 (0, 52),
 (0, 54),
 (0, 56),
 (0, 57),
 (0, 59),
 (0, 61),
 (0, 62),
 (0, 63),
 (0, 65),
 (0, 66),
 (0, 67),
 (0, 68),
 (0, 69),
 (0, 70),
 (0, 71),
 (0, 72),
 (0, 73),
 (0, 75),
 (0, 76),
 (0, 77),
 (0, 79),
 (0, 80),
 (0, 81),
 (0, 82),
 (0, 83),
 (0, 85),
 (0, 86),
 (0, 87),
 (0, 88),
 (0, 89),
 (0, 90),
 (0, 91),
 (0, 92),
 (0, 93),
 (0, 95),
 (0, 97),
 (0, 99),
 (0, 100),
 (0, 101),
 (0, 102),
 (0, 103),
 (0, 104),
 (0, 105),
 (0, 106),
 (0, 107),
 (0, 108),
 (0, 109),
 (0, 111),
 (0, 113),
 (0, 114),
 (0, 115),
 (0, 117),
 (0, 118),
 (0, 119),
 (0, 120),
 (0, 121),
 (0, 122),
 (0, 123),
 (0, 124),
 (0,

In [65]:
remainder_array=graph-A.residual

In [66]:
remainder_array[4,:]

<1x4002 sparse matrix of type '<class 'numpy.int64'>'
	with 0 stored elements in Compressed Sparse Row format>

In [67]:
from utils.flow import _make_edge_pointers

In [68]:
rev_edge_ptr, tails = _make_edge_pointers(remainder_array)

In [69]:
edge_ptr=remainder_array.indptr
capacities=remainder_array.data
heads=remainder_array.indices

In [70]:
tails

array([   0,    0,    0, ..., 4001, 4001, 4001], dtype=int32)

In [71]:
source=0
sink=n_1+n_2+1
ITYPE = np.int32

In [77]:
len(A.residual.indptr)

4003

In [72]:
n_verts = edge_ptr.shape[0] - 1
n_edges = capacities.shape[0]
ITYPE_MAX = np.iinfo(ITYPE).max

# Our result array will keep track of the flow along each edge
flow = np.zeros(n_edges, dtype=ITYPE)
# flow = A.residual

# Create a circular queue for breadth-first search. Elements are
# popped dequeued at index start and queued at index end.
q = np.empty(n_verts, dtype=ITYPE)

# Create an array indexing predecessor edges
pred_edge = np.empty(n_verts, dtype=ITYPE)

# cdef bint path_found
# cdef ITYPE_t cur, df, t, e, edge, k

# While augmenting paths from source to sink exist
for k in range(n_verts):
    pred_edge[k] = -1
path_edges = []
# Reset queue to consist only of source
q[0] = source
start = 0
end = 1
# While we have not found a path, and queue is not empty
path_found = False
while start != end and not path_found:
    # Pop queue
    cur = q[start]
    start += 1
    if start == n_verts:
        start = 0
    # Loop over all edges from the current vertex
    for e in range(edge_ptr[cur], edge_ptr[cur + 1]):
        t = heads[e]
        if pred_edge[t] == -1 and t != source and\
                capacities[e] > flow[e]:
            pred_edge[t] = e
            path_edges.append((cur,t))
            if t == sink:
                path_found = True
                break
            # Push to queue
            q[end] = t
            end += 1
            if end == n_verts:
                end = 0

/home/abhagoji/.conda/envs/pytor3/lib/python3.6/site-packages/scipy/sparse/compressed.py:287: SparseEfficiencyWarning: Comparing a sparse matrix with a scalar greater than zero using < is inefficient, try using >= instead.
  warn(bad_scalar_msg, SparseEfficiencyWarning)


ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all().

In [28]:
path_edges

[(0, 1),
 (0, 2),
 (0, 3),
 (0, 4),
 (0, 7),
 (0, 8),
 (0, 10),
 (0, 12),
 (0, 13),
 (0, 15),
 (0, 17),
 (0, 18),
 (0, 19),
 (0, 20),
 (0, 21),
 (0, 22),
 (0, 24),
 (0, 25),
 (0, 26),
 (0, 27),
 (0, 29),
 (0, 30),
 (0, 31),
 (0, 32),
 (0, 34),
 (0, 35),
 (0, 36),
 (0, 37),
 (0, 38),
 (0, 40),
 (0, 41),
 (0, 42),
 (0, 43),
 (0, 44),
 (0, 45),
 (0, 47),
 (0, 48),
 (0, 49),
 (0, 50),
 (0, 52),
 (0, 54),
 (0, 56),
 (0, 57),
 (0, 59),
 (0, 61),
 (0, 62),
 (0, 63),
 (0, 65),
 (0, 66),
 (0, 67),
 (0, 68),
 (0, 69),
 (0, 70),
 (0, 71),
 (0, 72),
 (0, 73),
 (0, 75),
 (0, 76),
 (0, 77),
 (0, 79),
 (0, 80),
 (0, 81),
 (0, 82),
 (0, 83),
 (0, 85),
 (0, 86),
 (0, 87),
 (0, 88),
 (0, 89),
 (0, 90),
 (0, 91),
 (0, 92),
 (0, 93),
 (0, 95),
 (0, 97),
 (0, 99),
 (0, 100),
 (0, 101),
 (0, 102),
 (0, 103),
 (0, 104),
 (0, 105),
 (0, 106),
 (0, 107),
 (0, 108),
 (0, 109),
 (0, 111),
 (0, 113),
 (0, 114),
 (0, 115),
 (0, 117),
 (0, 118),
 (0, 119),
 (0, 120),
 (0, 121),
 (0, 122),
 (0, 123),
 (0, 124),
 (0,

In [30]:
path_edges[50]==B.path_edges[50]

True

In [ ]:
import scipy
nz_flow_tuple_1=scipy.sparse.find(remainder_array)

In [ ]:
nz_flow_tuple

In [ ]:
nz_flow_tuple_1=scipy.sparse.find(remainder_array)

In [ ]:
gz_idx=np.where(nz_flow_tuple[2]>0)

gz_row_idx=nz_flow_tuple[0][gz_idx]
gz_col_idx=nz_flow_tuple[1][gz_idx]

In [ ]:
len(gz_idx[0])

In [ ]:
gz_row_idx

In [ ]:
gz_col_idx

In [ ]:
slice_row=np.unique(gz_row_idx)

In [ ]:
slice_col=np.unique(gz_col_idx)

In [ ]:
overall_slice=np.union1d(slice_row,slice_col)

In [ ]:
overall_slice

graph_rep_array=np.array(graph_rep)

graph_lvl_one=graph_rep_array[overall_slice]

graph_lvl_one[:,overall_slice]

In [ ]:
# Sort into lhs and rhs, rescale and run again; add termination condition for recursion down the graph; retain original numbering of array elements

In [ ]:
def set_classifier_prob_full_flow(top_level_vertices,n_1_curr,n_2_curr):
    for item in top_level_vertices:
        if item !=0 and item != sink_idx:
            classifier_probs[item-1,0]=n_1_curr/(n_1_curr+n_2_curr)
            classifier_probs[item-1,1]=n_2_curr/(n_1_curr+n_2_curr)

In [ ]:
def set_classifier_prob_no_flow(top_level_vertices):
    for item in top_level_vertices:
        if item !=0 and item != sink_idx:
            if item<=n_1:
                classifier_probs[item-1,0]=1
                classifier_probs[item-1,1]=0
            elif item>n_1:
                classifier_probs[item-1,0]=0
                classifier_probs[item-1,1]=1

In [ ]:
def graph_rescale(graph_rep_curr,top_level_indices):
    n_1_curr=len(np.where(top_level_indices<=n_1)[0])-1
    n_2_curr=len(np.where(top_level_indices>n_1)[0])-1
    # source rescale
    print(graph_rep_curr[0])
    graph_rep_curr[0,:]=graph_rep_curr[0,:]/n_2
    graph_rep_curr[0,:]*=n_2_curr
    print(graph_rep_curr[0])
    # bipartite graph edge scale
    graph_rep_curr[1:n_1_curr+1,:]=graph_rep_curr[1:n_1_curr+1,:]/(n_1*n_2)
    graph_rep_curr[1:n_1_curr+1,:]*=(n_1_curr*n_2_curr)
    # sink edges rescale
    graph_rep_curr[n_1_curr+1:,:]=graph_rep_curr[n_1_curr+1:,:]/n_1
    graph_rep_curr[n_1_curr+1:,:]*=n_1_curr
    return graph_rep_curr,n_1_curr,n_2_curr

In [ ]:
def find_flow_and_split(top_level_indices):
    top_level_indices_1=None
    top_level_indices_2=None
    #Create subgraph from index array provided
    graph_rep_curr = graph_rep_array[top_level_indices]
    graph_rep_curr = graph_rep_curr[:,top_level_indices]
    graph_rep_curr,n_1_curr,n_2_curr = graph_rescale(graph_rep_curr,top_level_indices)
    graph_curr=csr_matrix(graph_rep_curr)
    flow_curr = maximum_flow(graph_curr,0,len(top_level_indices)-1)
    # Checking if full flow occurred, so no need to split
    if flow_curr.flow_value==n_1_curr*n_2_curr:
        set_classifier_prob_full_flow(top_level_indices,n_1_curr,n_2_curr)
        return top_level_indices_1,top_level_indices_2, flow_curr
    elif flow_curr.flow_value==0:
        set_classifier_prob_no_flow(top_level_indices)
        return top_level_indices_1,top_level_indices_2, flow_curr
    # Finding remaining capacity edges
    edge_list_curr=flow_curr.path_edges
#     print(edge_list_curr)
    gz_idx = []
    for item in edge_list_curr:
        gz_idx.append(item[0])
        gz_idx.append(item[1])
    if len(gz_idx)>0:
        gz_idx=np.array(gz_idx)
        gz_idx_unique=np.unique(gz_idx)
        top_level_gz_idx=top_level_indices[gz_idx_unique]
        top_level_gz_idx=np.insert(top_level_gz_idx,len(top_level_gz_idx),sink_idx)
        top_level_indices_1=top_level_gz_idx
    else:
        top_level_gz_idx=np.array([0,sink_idx])
    # Indices without flow
    top_level_z_idx=np.setdiff1d(top_level_indices,top_level_gz_idx)
    if len(top_level_z_idx)>0:
        # Add source and sink back to zero flow idx array
        top_level_z_idx=np.insert(top_level_z_idx,0,0)
        top_level_z_idx=np.insert(top_level_z_idx,len(top_level_z_idx),sink_idx)
        top_level_indices_2=top_level_z_idx
    
    return top_level_indices_1,top_level_indices_2, flow_curr

In [ ]:
import time
import queue
q = queue.Queue()
time1=time.time()
# Initial graph indices
q.put(np.arange(n_1+n_2+2))
sink_idx=n_1+n_2+1
count=0
classifier_probs=np.zeros((n_1+n_2,2))
while not q.empty():
    print(q.qsize())
    curr_idx_list=q.get()
    print(q.qsize())
    list_1, list_2, flow_curr=find_flow_and_split(curr_idx_list)
    print(list_1,list_2,flow_curr.flow_value)
#     if list_1 is None and list_2 is None:
#         trial_var=set_global_probs(flow_curr,curr_idx_list)
    if list_1 is not None:
#         flow_possible_1=check_flow_possible(list_1)
        q.put(list_1)
    if list_2 is not None:
#         flow_possible_2=check_flow_possible(list_2)
        q.put(list_2)
time2=time.time()

In [ ]:
classifier_probs

In [ ]:
loss = 0.0
for i in range(len(classifier_probs)):
    if i<n_1:
        loss+=np.log(classifier_probs[i][0])
    elif i>=n_1:
        loss+=np.log(classifier_probs[i][1])
loss=-1.0*loss/len(classifier_probs)
print(loss)

In [ ]:
print('Time taken: %s' % (time2-time1))

# Directly solve the convex program

In [94]:
from cvxopt import solvers, matrix, spdiag, log, mul, sparse, spmatrix

In [95]:
v=n_1+n_2

In [96]:
def minll(G,h,p):
    m,v_in=G.size
    def F(x=None,z=None):
        if x is None:
            return 0, matrix(1.0,(v,1))
        if min(x)<=0.0:
            return None
        f = -sum(mul(p,log(x)))
        Df = mul(p.T,-(x**-1).T)
        if z is None:
            return f,Df
        # Fix the Hessian
        H = spdiag(z[0]*mul(p,x**-2))
        return f,Df,H
    return solvers.cp(F,G=G,h=h)

In [97]:
num_edges=len(np.where(edge_matrix==1)[0])
num_edges

35055

In [98]:
edges=np.where(edge_matrix==1)
edges

(array([   4,    4,    5, ..., 1989, 1989, 1989]),
 array([ 838, 1278,   37, ..., 1767, 1799, 1988]))

In [99]:
incidence_matrix=np.zeros((num_edges,v))

In [100]:
for i in range(num_edges):
    j1=edges[0][i]
    j2=edges[1][i]+(n_1-1)
    incidence_matrix[i,j1]=1
    incidence_matrix[i,j2]=1
incidence_matrix

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [101]:
len(np.where(incidence_matrix!=0)[0])

70110

In [102]:
G_in=np.vstack((incidence_matrix,np.eye(v)))

In [103]:
G_in.shape

(39055, 4000)

In [104]:
from scipy.sparse import coo_matrix
G_in_sparse_np=coo_matrix(G_in)

In [105]:
G_in_sparse_np.nonzero()[0]

array([    0,     0,     1, ..., 39052, 39053, 39054], dtype=int32)

In [106]:
h_in=np.ones((num_edges+v,1))

In [107]:
p=(1.0/v)*np.ones((v,1))

In [108]:
solvers.options['maxiters']=10000

In [ ]:
# Does not work when the matrix is really large
G_in_sparse=sparse(matrix(G_in))
G_in_dense=matrix(G_in)

In [109]:
G_in_sparse=spmatrix(1.0,G_in_sparse_np.nonzero()[0],G_in_sparse_np.nonzero()[1])

In [110]:
time3=time.time()
output=minll(G_in_sparse,matrix(h_in),matrix(p))
time4=time.time()

     pcost       dcost       gap    pres   dres
 0:  0.0000e+00  3.5055e+04  4e+04  1e+00  1e+00
 1:  9.8929e-01 -3.9487e+02  4e+02  1e-02  1e-02
 2:  1.1865e+00 -3.7430e+02  4e+02  1e-02  1e-02
 3:  1.3765e+00 -3.5420e+02  4e+02  1e-02  9e-03
 4:  3.6088e+00 -8.2938e+00  1e+01  7e-04  2e-04
 5:  2.9031e+00  2.8341e+00  4e-01  8e-04  1e-04
 6:  2.2116e+00  2.4973e+00  2e-02  8e-04  8e-05
 7:  1.5166e+00  1.8216e+00  2e-03  8e-04  4e-05
 8:  8.2346e-01  1.1299e+00  3e-04  8e-04  2e-05
 9:  5.6476e-01  8.1286e-01  2e-04  7e-04  1e-05
10:  5.6178e-01  8.0890e-01  2e-04  7e-04  1e-05
11:  5.5850e-01  8.0454e-01  2e-04  6e-04  1e-05
12:  5.5006e-01  7.9339e-01  2e-04  6e-04  1e-05
13:  5.4697e-01  7.8931e-01  2e-04  6e-04  1e-05
14:  5.4260e-01  7.8354e-01  2e-04  6e-04  1e-05
15:  5.3755e-01  7.7691e-01  2e-04  6e-04  1e-05
16:  5.3209e-01  7.6974e-01  2e-04  6e-04  1e-05
17:  5.2627e-01  7.6210e-01  2e-04  6e-04  1e-05
18:  5.2009e-01  7.5402e-01  2e-04  6e-04  1e-05
19:  5.1399e-01  7.46

In [ ]:
print(time4-time3)

In [ ]:
print(output['primal objective'])

In [111]:
print(output)

{'status': 'optimal', 'x': <4000x1 matrix, tc='d'>, 'y': <0x1 matrix, tc='d'>, 'znl': <0x1 matrix, tc='d'>, 'zl': <39055x1 matrix, tc='d'>, 'snl': <0x1 matrix, tc='d'>, 'sl': <39055x1 matrix, tc='d'>, 'gap': 2.8562354314984288e-08, 'relative gap': 1.143124588506504e-07, 'primal objective': 0.24986176092637247, 'dual objective': 0.24986212878423947, 'primal slack': 5.602880612567375e-13, 'dual slack': 5.680735368928723e-13, 'primal infeasibility': 1.0438582990533191e-09, 'dual infeasibility': 2.4601502235557537e-11}
